In [ ]:
import pandas as pd
from openai import OpenAI
import pickle
import time
from tqdm.notebook import tqdm

In [ ]:
df_test = pd.read_csv('dodiom_it_test.csv')
df_train = pd.read_csv('dodiom_it_train.csv')

In [ ]:
df_test

In [ ]:
idioms = df_test["submission"].tolist()

In [ ]:
client = OpenAI(api_key="")

In [ ]:
for t in tqdm(range(len(df_test))):  # Loop over the range of idioms
    i = df_test['submission'].iloc[t]  # Access each idiom directly using the loop index
    if len(df_train[(df_train['category'] == 'nonidiom') & (df_train['idiom'] == df_test['idiom'].iloc[t])]) == 0:
        print("Cannot find the idiom in the training set: ", df_test['idiom'].iloc[t])

In [ ]:
idioms

In [ ]:
results = []
for t in tqdm(range(len(df_test))):  # Loop over the range of idioms
    i = df_test['submission'].iloc[t]  # Access each idiom directly using the loop index

    literals = []
    for m in range(3):
        if len(df_train[(df_train['category'] == 'nonidiom') & (df_train['idiom'] == df_test['idiom'].iloc[t])]) == 0:
            print("Cannot find the idiom in the training set: ", df_test['idiom'].iloc[t])
            text = df_train[(df_train['category'] == 'nonidiom')].sample(1)['submission'].iloc[0]
        else:
            text = df_train[(df_train['category'] == 'nonidiom') & (df_train['idiom'] == df_test['idiom'].iloc[t])].sample(1)['submission'].iloc[0]
        literals.append(text)

    figuratives = []
    for m in range(3):

        text = df_train[(df_train['category'] == 'idiom') & (df_train['idiom'] == df_test['idiom'].iloc[t])].sample(1)['submission'].iloc[0]
        figuratives.append(text)

    completion = client.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
    {"role": "system", "content":
        "Il tuo compito è quello di essere un modello linguistico che parla italiano. Rispondi seguendo esclusivamente le istruzioni fornite e non scrivere nient'altro."
    },
    {"role": "user", "content":
        "Voglio che tu identifichi se la frase qui sotto contiene o meno un'espressione idiomatica. Se la frase contiene un'espressione idiomatica, scrivi solo '1'. Se non contiene, scrivi solo '0'. Fornisci solo '0' o '1' come risposta, e nient'altro. Frase:\n"
        + literals[0] + ": 0\n" + literals[1] + ": 0\n" + literals[2] + ": 0\n"
        + figuratives[0] + ": 1\n" + figuratives[1] + ": 1\n" + figuratives[2] + ": 1\n"
        + i + ": ?"
    }
    ]
    )
    print(completion.choices[0].message.content)

    while completion.choices[0].message.content == "?" or len(completion.choices[0].message.content) > 1:
        completion = client.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
      {"role": "system", "content":
        "Il tuo compito è quello di essere un modello linguistico che parla italiano. Rispondi seguendo esclusivamente le istruzioni fornite e non scrivere nient'altro."
    },
    {"role": "user", "content":
        "Voglio che tu identifichi se la frase qui sotto contiene o meno un'espressione idiomatica. Se la frase contiene un'espressione idiomatica, scrivi solo '1'. Se non contiene, scrivi solo '0'. Fornisci solo '0' o '1' come risposta, e nient'altro. Frase:\n"
        + literals[0] + ": 0\n" + literals[1] + ": 0\n" + literals[2] + ": 0\n"
        + figuratives[0] + ": 1\n" + figuratives[1] + ": 1\n" + figuratives[2] + ": 1\n"
        + i + ": ?"
    }
    ]
    )

    results.append(completion)  # Append the result to the list

In [ ]:
with open('gpt-4-turbo_1_it.pkl', 'wb') as f:
      pickle.dump(results, f)

In [ ]:
labels = df_test["category"].tolist()

In [ ]:
df = pd.DataFrame([i.choices[0].message.content for i in results])
df.to_csv('results_gpt-4-turbo_it.csv', index=False)

In [ ]:
df

In [ ]:
for i in [i for i, x in enumerate(df[0].tolist()) if x == "1\n"]:
    df[0][i] = "1"

In [ ]:
df

In [ ]:
gpt_labels = [int(i) for i in df.iloc[:, 0].tolist()]


In [ ]:
human_labels = [1 if i == "idiom" else 0 for i in labels]

In [ ]:
# make value 3 as 0 in gpt_labels
for i in [i for i, x in enumerate(gpt_labels) if x == 3]:
    gpt_labels[i] = 0

In [ ]:
[i for i in gpt_labels if i != 0 and i != 1]


In [ ]:
human_labels

In [ ]:
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(human_labels, gpt_labels)

print(report)


In [ ]:
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(human_labels, gpt_labels, digits=4)

print(report)
